In [1]:
!pip install fastavro

In [1]:
!pyspark --version

22/06/19 18:46:13 WARN Utils: Your hostname, vplentz-computer resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface wlp2s0)
22/06/19 18:46:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.2.1
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user hgao on 2022-01-20T19:26:14Z
Revision 4f25b3f71238a00508a356591553f2dfa89f8290
Url https://github.com/apache/spark
Type --help for more information.


In [1]:
# za.co.absa" %% "abris" % "4.0.1
import os
kafka_deps = 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1'
confluent_schema_deps = 'io.confluent:kafka-schema-registry-client:7.1.1,' + 'io.confluent:kafka-avro-serializer:7.1.1'
abris_deps = 'za.co.absa:abris_2.12:6.3.0' 
#jars_path = ",".join(['/home/jovyan/work/pyspark_structured_streaming/jars/jar_files'+file for file in os.listdir('/home/jovyan/work/pyspark_structured_streaming/jars/jar_files/')])
spark_deps  = f"--repositories https://packages.confluent.io/maven --packages {kafka_deps},{confluent_schema_deps},{abris_deps} pyspark-shell"
print(spark_deps)
os.environ['PYSPARK_SUBMIT_ARGS'] = spark_deps

--repositories https://packages.confluent.io/maven --packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,io.confluent:kafka-schema-registry-client:7.1.1,io.confluent:kafka-avro-serializer:7.1.1,za.co.absa:abris_2.12:6.3.0 pyspark-shell


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, length

In [3]:
spark_context = SparkSession.builder.appName('processingBots').getOrCreate()

In [4]:
from pyspark import SparkContext
from pyspark.sql.column import Column, _to_java_column

def from_avro(col, config):
    """
    avro deserialize

    :param col (PySpark column / str): column name "key" or "value"
    :param config (za.co.absa.abris.config.FromAvroConfig): abris config, generated from abris_config helper function
    :return: PySpark Column
    """
    jvm_gateway = SparkContext._active_spark_context._gateway.jvm
    abris_avro = jvm_gateway.za.co.absa.abris.avro

    return Column(abris_avro.functions.from_avro(_to_java_column(col), config))

def from_avro_abris_config(config_map, topic, is_key):
    """
    Create from avro abris config with a schema url

    :param config_map (dict[str, str]): configuration map to pass to deserializer, ex: {'schema.registry.url': 'http://localhost:8081'}
    :param topic (str): kafka topic
    :param is_key (bool): boolean
    :return: za.co.absa.abris.config.FromAvroConfig
    """
    jvm_gateway = SparkContext._active_spark_context._gateway.jvm
    scala_map = jvm_gateway.PythonUtils.toScalaMap(config_map)

    return jvm_gateway.za.co.absa.abris.config \
        .AbrisConfig \
        .fromConfluentAvro() \
        .downloadReaderSchemaByLatestVersion() \
        .andTopicNameStrategy(topic, is_key) \
        .usingSchemaRegistry(scala_map)

In [5]:
starting_offsets = 'earliest'

wikipedia_df = spark_context.readStream.format('kafka')\
    .option('kafka.bootstrap.servers', 'localhost:12091')\
    .option('subscribe', 'wikipedia.parsed')\
    .option("startingOffsets", starting_offsets)\
    .load()

In [12]:
!echo ${PWD}

/home/jovyan/work/pyspark_structured_streaming


In [7]:
wikipedia_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
import fastavro
import io

In [16]:
confluent_config = {
    'schema.registry.url':'https://localhost:8085',
    #"schema.registry.ssl.certificate.location": "/home/jovyan/work/scripts/security/snakeoil-ca-1.crt",
    "schema.registry.ssl.truststore.location" :"/home/jovyan/work/scripts/security/kafka.schemaregistry.truststore.jks",
    "schema.registry.ssl.truststore.password": 'confluent',
    "schema.registry.ssl.keystore.location": '/home/jovyan/work/scripts/security/kafka.schemaregistry.keystore.jks',
    "schema.registry.ssl.keystore.password" : "confluent",
    "basic.auth.credentials.source": "USER_INFO",
    "basic.auth.user.info": "superUser:superUser"
} 
from_avro_abris_settings = from_avro_abris_config(confluent_config, 'wikipedia.parsed', False)
wikipedia_df = wikipedia_df.withColumn('value', from_avro(col('value'), from_avro_abris_settings))

In [17]:
wikipedia_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- bot: boolean (nullable = true)
 |    |-- comment: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- length: struct (nullable = true)
 |    |    |-- new: long (nullable = true)
 |    |    |-- old: long (nullable = true)
 |    |-- log_action: string (nullable = true)
 |    |-- log_action_comment: string (nullable = true)
 |    |-- log_id: long (nullable = true)
 |    |-- log_type: string (nullable = true)
 |    |-- meta: struct (nullable = false)
 |    |    |-- domain: string (nullable = true)
 |    |    |-- dt: timestamp (nullable = false)
 |    |    |-- id: string (nullable = false)
 |    |    |-- request_id: string (nullable = true)
 |    |    |-- stream: string (nullable = false)
 |    |    |-- uri: string (nullable = true)
 |    |-- minor: boolean (nullable = true)
 |    |-- namespace: long (nullable = true)
 |    |-- parsedcomment: string (nullable = true)
 |    |-- patrolle

In [18]:
wikipedia_df.writeStream.format('console').start()

In [14]:
wikipedia_df.filter(col('value.bot') != True).\
    .withColumn('BYTECHANGE', (col('length.new') - col('length.old'))).
    .select('*').writeStream.format('kafka')\
    .trigger(processingTime='5 minutes')\
    .option("kafka.bootstrap.servers", "localhost:8091") \
    .option("checkpointLocation", "../checkpoints/") \
        .option("topic", "WIKIPEDIABOT") \
    .outputMode("append") \
    .start()